In [ ]:
from itertools import product

import numpy as np
import xarray as xr

import pysqkit as sq

### Set some matplotlib parameters

### Define some utility functions

In [ ]:
transmon = sq.qubits.SimpleTransmon(
    label='transmon', 
    max_freq = 6, 
    anharm = -0.3, 
    dim_hilbert = 5
)

In [ ]:
fluxonium = sq.qubits.Fluxonium(
    label='fluxonium',
    joseph_energy=5.900, 
    charge_energy=0.975, 
    induct_energy=0.455,
    flux = 0.5
)
fluxonium.diagonalize_basis(5)

Add the drive (generally you are able to add it after they're coupled. Currently the dimension specified in the Qobj are actually not correct. I'll fix that soon, so basically for now add them before coupling (or fix dimensions manually)

In [ ]:
fluxonium.add_drive(
    sq.drives.microwave_drive,
    label='cz_drive',
    pulse_shape=sq.drives.pulse_shapes.gaussian_top
)

In [ ]:
system = transmon.couple_to(fluxonium, coupling=sq.couplers.capacitive_coupling, strength=0.2)

you can look at free parameters to be fixed

In [ ]:
system['fluxonium'].drives[0].free_params

if you know phase is always going to be 0, you can go ahead and fix it to the default parameters

In [ ]:
system['fluxonium'].drives[0].set_params(phase=0)

In [ ]:
system['fluxonium'].drives[0].free_params

we see it's been fixed to a value. The other parameters can also be fixed like this or provided when you're evaluating the pulse before doing the Lindblad simulation

In [ ]:
times = np.linspace(0, 40, 2000)
amp = 0.2
freq = 0.5

hamil = system.hamiltonian(as_qobj=True)
drive_hamils = [drive.hamiltonian(as_qobj=True) for qubit in system for drive in qubit.drives]
pulses = [drive.eval_pulse(time=times, amp=amp, freq=freq) for qubit in system for drive in qubit.drives]

In [ ]:
energy, state = system.state('00', as_qobj=True)

In [ ]:
result = sq.solvers.integrate(times, state, hamil, drive_hamils, pulses, jump_op=None, solver='mesolve')

In [ ]:
pulses

In [ ]:
pulses[0]